In [3]:
import os
import xml.etree.ElementTree as ET
import numpy as np
from PIL import Image

In [2]:


def create_original_sized_mask(cropped_mask, original_image_path, extended_coords, output_path):
    # Open the original image to get its dimensions
    original_image = Image.open(original_image_path)
    original_width, original_height = original_image.size

    # Create a full zero mask with the original image dimensions
    original_mask = Image.new('L', (original_width, original_height), 0)  # 'L' mode for single channel (grayscale)

    # Ensure the cropped mask is single-channel
    if cropped_mask.mode != 'L':
        cropped_mask = cropped_mask.convert('L')

    # Convert images to numpy arrays
    cropped_mask_array = np.array(cropped_mask)
    original_mask_array = np.array(original_mask)

    y_min, y_max, x_min, x_max = extended_coords

    # Ensure the mask is placed within the bounds of the original mask
    original_mask_array[y_min:y_max, x_min:x_max] = cropped_mask_array

    # Convert the array back to an image
    original_mask = Image.fromarray(original_mask_array)

    # Save the mask
    original_mask.save(output_path)

def get_extended_coordinates(annotation_path, image_width, image_height):
    try:
        # Parse the annotation XML file
        tree = ET.parse(annotation_path)
        root = tree.getroot()

        # Get the bounding box coordinates
        bbox = root.find('object').find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # Calculate the extended cropping coordinates using the provided logic
        x, y, w, h = xmin, ymin, xmax - xmin, ymax - ymin
        y_min = int(y - (h / 4))
        y_max = int(y + h + (h / 4))
        x_min = int(x - (w / 4))
        x_max = int(x + w + (w / 4))

        # Ensure the coordinates are within the image bounds
        y_min = max(0, y_min)
        y_max = min(image_height, y_max)
        x_min = max(0, x_min)
        x_max = min(image_width, x_max)

        return y_min, y_max, x_min, x_max

    except AttributeError:
        # Log the filename if the expected XML structure is not found
        with open('error_log.txt', 'a') as f:
            f.write(f"Error processing {annotation_path}: 'object' tag not found\n")
        return None
    except Exception as e:
        # Handle other exceptions gracefully if needed
        with open('error_log.txt', 'a') as f:
            f.write(f"Error processing {annotation_path}: {str(e)}\n")
        return None

def process_images(image_dir, annotation_dir, cropped_mask_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get sorted list of files in annotation directory
    annotation_files = sorted(os.listdir(annotation_dir))

    # Process each annotation file
    for annotation_filename in annotation_files:
        if annotation_filename.endswith('.xml'):
            base_name = annotation_filename[:-4]
            image_path = os.path.join(image_dir, base_name + '.jpg')
            annotation_path = os.path.join(annotation_dir, annotation_filename)
            cropped_mask_path = os.path.join(cropped_mask_dir, 'croped_mask' + base_name + '.png')  # Adjusted mask filename

            if not os.path.exists(image_path) or not os.path.exists(cropped_mask_path):
                print(f"Missing corresponding image or mask for {annotation_filename}")
                continue

            # Open the original image to get its dimensions
            original_image = Image.open(image_path)
            original_width, original_height = original_image.size

            # Get the extended coordinates
            extended_coords = get_extended_coordinates(annotation_path, original_width, original_height)

            if extended_coords:
                # Load the cropped mask
                cropped_mask = Image.open(cropped_mask_path)

                # Create and save the original-sized mask
                output_path = os.path.join(output_dir, base_name + '_original_size_mask.png')
                create_original_sized_mask(cropped_mask, image_path, extended_coords, output_path)
                print(f"Original-sized mask saved: {output_path}")

image_dir = '/mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/train/img'
annotation_dir = '/mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/train/xml'
cropped_mask_dir = '/mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/croped_mask'
output_dir = '/mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/original_sized_mask'

process_images(image_dir, annotation_dir, cropped_mask_dir, output_dir)


Original-sized mask saved: /mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/original_sized_mask/00001_original_size_mask.png
Original-sized mask saved: /mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/original_sized_mask/00002_original_size_mask.png
Original-sized mask saved: /mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/original_sized_mask/00003_original_size_mask.png
Original-sized mask saved: /mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/original_sized_mask/00004_original_size_mask.png
Original-sized mask saved: /mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/original_sized_mask/00005_original_size_mask.png
Original-sized mask saved: /mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/original_sized_mask/00006_original_size_mask.png
Original-sized mask saved: /mnt/DATA/dronetracking/Abhin/segement&detect/detection_dataset/original_sized_mask/00007_original_size_mask.png
Original-sized mask 

In [4]:
original_image = Image.open("/mnt/DATA/dronetracking/Abhin/final_composition/paper_work_sample/LaSot/00000146.jpg") 
original_width, original_height = original_image.size

cropped_mask = Image.open("/mnt/DATA/dronetracking/Abhin/final_composition/paper_work_sample/LaSot/u-2-net/croped_mask146_cropped.png")

# Calculate the extended cropping coordinates using the provided logic
x, y, w, h = 653,363,105,53
y_min = int(y - (h / 4))
y_max = int(y + h + (h / 4))
x_min = int(x - (w / 4))
x_max = int(x + w + (w / 4))

# Ensure the coordinates are within the image bounds
y_min = max(0, y_min)
y_max = min(original_height, y_max)
x_min = max(0, x_min)
x_max = min(original_width, x_max)

# Create a full zero mask with the original image dimensions
original_mask = Image.new('L', (original_width, original_height), 0)  # 'L' mode for single channel (grayscale)

# Ensure the cropped mask is single-channel
if cropped_mask.mode != 'L':
    cropped_mask = cropped_mask.convert('L')

# Convert images to numpy arrays
cropped_mask_array = np.array(cropped_mask)
original_mask_array = np.array(original_mask)


# Ensure the mask is placed within the bounds of the original mask
original_mask_array[y_min:y_max, x_min:x_max] = cropped_mask_array

# Convert the array back to an image
original_mask = Image.fromarray(original_mask_array)

# Save the mask
original_mask.save("/mnt/DATA/dronetracking/Abhin/final_composition/paper_work_sample/LaSot/u-2-net/croped_mask/original_size_mask.png")